# Data Aggregation & Summaries

Use this Jupyter Notebook to take our initial dataset(s) and generate aggregate/reformatted/summarized CSVs usable by our frontend.

In [ ]:
# imports and initial variables
import pandas as pd
dataset_name = "census_tracts.csv"
dataset_df = pd.read_csv(dataset_name)